In [7]:
from scipy.spatial.distance import cdist
import os
import sys
import glob
import random
import numpy as np
import logging
import json
import torch
import math
#from pathlib import Path
import matplotlib.pyplot as plt

In [8]:
#####################################################################################
# Load poses
#####################################################################################

ch = logging.StreamHandler(sys.stdout)
logging.getLogger().setLevel(logging.INFO)
logging.basicConfig(format='%(asctime)s %(message)s',
                    datefmt='%m/%d %H:%M:%S',
                    handlers=[ch])
logging.basicConfig(level=logging.INFO, format="")

def _pad_tensors_to_max_len( tensor, max_length,tokenizer):
    # If PAD token is not defined at least EOS token has to be defined
    pad_token_id = (
        tokenizer.pad_token_id if tokenizer.pad_token_id is not None else tokenizer.eos_token_id
    )
    tensor[tensor == -100] = tokenizer.pad_token_id
    padded_tensor = pad_token_id * torch.ones(
        (tensor.shape[0], max_length), dtype=tensor.dtype, device=tensor.device
    )
    padded_tensor[:, : tensor.shape[-1]] = tensor
    return padded_tensor


def transfrom_cam2velo(Tcam):
    R = np.array([7.533745e-03, -9.999714e-01, -6.166020e-04, 1.480249e-02, 7.280733e-04,
                  -9.998902e-01, 9.998621e-01, 7.523790e-03, 1.480755e-02
                  ]).reshape(3, 3)
    t = np.array([-4.069766e-03, -7.631618e-02, -2.717806e-01]).reshape(3, 1)
    cam2velo = np.vstack((np.hstack([R, t]), [0, 0, 0, 1]))

    return Tcam @ cam2velo


def load_poses_from_txt(file_name):
    """
    Modified function from: https://github.com/Huangying-Zhan/kitti-odom-eval/blob/master/kitti_odometry.py
    """
    f = open(file_name, 'r')
    s = f.readlines()
    f.close()
    transforms = {}
    positions = []
    for cnt, line in enumerate(s):
        P = np.eye(4)
        line_split = [float(i) for i in line.split(" ") if i != ""]
        withIdx = len(line_split) == 13
        for row in range(3):
            for col in range(4):
                P[row, col] = line_split[row*4 + col + withIdx]
        if withIdx:
            frame_idx = line_split[0]
        else:
            frame_idx = cnt
        transforms[frame_idx] = transfrom_cam2velo(P)
        positions.append([P[0, 3], P[2, 3], P[1, 3]])
    return transforms, np.asarray(positions)


#####################################################################################
# Load timestamps
#####################################################################################


def load_timestamps(file_name):
    # file_name = data_dir + '/times.txt'
    file1 = open(file_name, 'r+')
    stimes_list = file1.readlines()
    s_exp_list = np.asarray([float(t[-4:-1]) for t in stimes_list])
    times_list = np.asarray([float(t[:-2]) for t in stimes_list])
    times_listn = [times_list[t] * (10**(s_exp_list[t]))
                   for t in range(len(times_list))]
    file1.close()
    return times_listn

In [11]:
eval_seq = 0
log3dnet_dir='/lustre/fswork/projects/rech/dki/ujo91el/code/these_place_reco/LoGG3D-Net/'
revisit_criteria=3
not_revisit_criteria=20
skip_time=30
revisit_json_file = 'is_revisit_D-{}_T-{}.json'.format(
int(revisit_criteria), int(skip_time))
cd_thresh_min=0.001
cd_thresh_max=5
num_thresholds=1000
num_beams = 10
## ==== Kitti =====
kitti_dir= os.getenv('WORK') + '/datas/datasets/'
eval_seq = '%02d' % eval_seq
sequence_path = kitti_dir + 'sequences/' + eval_seq + '/'
_, positions_database = load_poses_from_txt(sequence_path + 'poses.txt')

min_bbox = np.min(positions_database,0) 
positions_database = positions_database - min_bbox

timestamps = load_timestamps(sequence_path + 'times.txt')
revisit_json_dir = os.path.join(os.path.dirname(__file__), '/config/kitti_tuples/')
revisit_json = json.load(open(log3dnet_dir + revisit_json_dir + revisit_json_file, "r"))
is_revisit_list = revisit_json[eval_seq]

In [22]:
c = 0
for i in range (len(revisit_json)):
    c += len(revisit_json['%02d' % i])
c

23201

In [25]:
revisit_criteria = 3
revisit_criteria_f = 20
skip_time = 0
tuple_dir = "/lustre/fsn1/worksf/projects/rech/dki/ujo91el/datas/datasets/sequences/00/"

kitti_3m_json = 'positive_sequence_D-{}_T-{}.json'.format(int(revisit_criteria), int(skip_time))
kitti_20m_json = 'positive_sequence_D-{}_T-{}.json'.format(int(revisit_criteria_f), int(skip_time))
    
dict_3m = json.load(open(tuple_dir + kitti_3m_json, "r"))
dict_20m = json.load(open(tuple_dir + kitti_20m_json, "r"))

In [29]:
len(dict_3m)

11